In [1]:
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET 
import os
# xml = BeautifulSoup('output.xml','xml.parser')
xml2 = ET.parse('output.xml')

In [2]:
index=0
snip_dict = {}
for dup_tag in xml2.getroot().findall('pmd-cpd/duplication'):
    for file_tag in dup_tag.findall('file'):
        val = file_tag.get('path')
        if val.endswith('snippets.java'):
            code = dup_tag.find('codefragment').text
            if val in snip_dict.keys():
                vl = snip_dict[val]
                vl.append(code)
                snip_dict[val]=vl
            else:
                vl = []
                vl.append(code)
                snip_dict[val]=vl
            


In [3]:
# l = list(snip_dict.keys())
# k = (snip_dict[l[0]])
# l[0]

In [4]:
import pandas as pd
columns = ['FilePath','Found in','% snippet in diff']

df = pd.DataFrame(columns=columns)
df

,FilePath,Found in,% snippet in diff


In [5]:
import subprocess
import shlex

def get_snip_id(snip_list, file_text):
    snip_id = -1
    index=-1
    for snip in snip_list:
        index+=1
        if file_text.find(snip)!=-1:
            if snip_id==-1:
                snip_id=index
            elif len(snip)>len(snip_list[snip_id]):
                snip_id=index
    return snip_id

for key in snip_dict.keys():
    act_fldr = key.replace('/home/kashivns/soft/./file-data/','').replace('/snippets.java','')
    dir_list = os.listdir(act_fldr)
    num_dir=0
    for d in dir_list:
        if os.path.isdir(os.path.join(act_fldr,d)):
            num_dir+=1
    for i in range(num_dir,1,-1):
        fd = os.path.join(act_fldr,str(i))
        fi = os.listdir(fd)[0]
        with open(os.path.join(fd,fi), 'r') as fil:
            data = fil.read()
        snp_ind = get_snip_id(snip_dict[key],data)
        if(snp_ind!=-1):
            if i==num_dir:
                continue
            fd2=os.path.join(act_fldr,str(i+1))
            fi2 = os.listdir(fd2)[0]
            with open(os.path.join(fd2,fi2), 'r') as fil:
                data2 = fil.read()
            sd1 = os.path.join(fd2,fi2)
            sd2 = os.path.join(fd,fi)
            d2 = snip_dict[key][snp_ind]
            os.system('./getDiff.sh '+sd1+' '+sd2)
            with open(os.path.join('temp.txt'), 'r') as fil:
                d1 = fil.read()
            percent_snip = len(d2)/len(d1)
            os.remove('temp.txt')
            row = {'FilePath': key, 'Found in': i, '% snippet in diff': percent_snip}
#             print(row)
            df = df.append(row, ignore_index=True)
            break

In [6]:
df

,FilePath,Found in,% snippet in diff
0,/home/kashivns/soft/./file-data/Android-Bootst...,4,0.504286
1,/home/kashivns/soft/./file-data/BloomReader/ap...,27,0.962445
2,/home/kashivns/soft/./file-data/BungeeChat2/sr...,4,0.057229
3,/home/kashivns/soft/./file-data/CEFCommon/src-...,7,4.478571
4,/home/kashivns/soft/./file-data/CastOffline/sr...,3,0.912424
...,...,...,...
90,/home/kashivns/soft/./file-data/superluminal2/...,2,0.057376
91,/home/kashivns/soft/./file-data/surveyor/app-s...,11,0.205615
92,/home/kashivns/soft/./file-data/test/src-main-...,16,0.292526
93,/home/kashivns/soft/./file-data/the-week-magaz...,3,0.418301


In [15]:
count=0
for i in range(len(df)):
    if df['% snippet in diff'][i]>0.2:
        count+=1
print(count)

57


In [12]:
df.to_csv('output1.csv')